In [18]:
# @title Célula 1: Instalação e Imports
!pip install -q google-generativeai feedparser

# @title Imports
import google.generativeai as genai
import os
import feedparser # Importe feedparser

In [19]:
# @title Célula 2: Configura a API Key do Google Gemini

from google.colab import userdata

try:
  GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
  if not GOOGLE_API_KEY:
      raise ValueError("GOOGLE_API_KEY não encontrada nos segredos do Colab.")
  genai.configure(api_key=GOOGLE_API_KEY)
  print("API Key configurada com sucesso.")
except Exception as e:
  print(f"Erro ao configurar a API Key: {e}")
  print("Certifique-se de ter adicionado sua GOOGLE_API_KEY nos segredos do Colab.")
  GOOGLE_API_KEY = None # Garante que a chave não está definida se houver erro

API Key configurada com sucesso.


In [20]:
# @title Célula 3: Configura o cliente da SDK do Gemini

if GOOGLE_API_KEY:
    # Escolha o modelo (experimente diferentes)
    model_name = 'gemini-2.0-flash' # Ou 'gemini-1.0-pro'

    generation_config = {
      "temperature": 0.9, # Experimente valores para criatividade
      "top_p": 0.95,
      "top_k": 60,
      "max_output_tokens": 256, # Limite o tamanho da resposta
    }

    safety_settings = [
      {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
      {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
      {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
      {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    ]

    model = genai.GenerativeModel(model_name=model_name,
                                  generation_config=generation_config,
                                  safety_settings=safety_settings)

    print(f"Modelo {model_name} configurado com sucesso.")
else:
    model = None
    print("Modelo não configurado devido à falta da API Key.")

Modelo gemini-2.0-flash configurado com sucesso.


In [21]:
# @title Célula 4: Módulo de Busca de Notícias (Usando RSS da Agência Brasil)

def buscar_noticias_economicas_rss(feed_url="https://agenciabrasil.ebc.com.br/rss/economia/feed.xml", max_noticias=3):
    """
    Busca notícias econômicas recentes de um feed RSS.

    Args:
        feed_url: URL do feed RSS da Agência Brasil - Economia.
        max_noticias: Número máximo de notícias a serem retornadas.

    Returns:
        Uma lista de strings, onde cada string é o conteúdo (Título + Resumo)
        de uma notícia, ou uma lista vazia em caso de erro.
    """
    print(f"Buscando notícias do feed RSS: {feed_url}")
    noticias_processadas = []

    try:
        # Usa feedparser para parsear o feed diretamente da URL
        feed = feedparser.parse(feed_url)

        # Verifica se houve algum erro ao parsear o feed
        if feed.bozo:
            print(f"Atenção: Ocorreu um erro ao parsear o feed RSS ({feed.bozo_exception}). Pode estar incompleto ou mal formatado.")
            # Você pode decidir retornar aqui ou tentar processar o que foi lido

        # Verifica se o feed contém entradas (notícias)
        if not feed.entries:
            print("Nenhuma entrada encontrada no feed RSS.")
            return []

        print(f"Encontradas {len(feed.entries)} entradas no feed. Processando as primeiras {max_noticias}...")

        # Itera sobre as entradas (notícias) do feed
        for i, entry in enumerate(feed.entries[:max_noticias]):
            # Extrai Título, Link e Resumo/Descrição de cada entrada
            # feedparser tenta normalizar 'summary' e 'description' para 'summary'
            titulo = entry.get('title', 'Sem Título')
            link = entry.get('link', 'Sem Link')
            resumo = entry.get('summary', entry.get('description', 'Sem Resumo')) # Tenta summary, depois description

            # Cria uma string combinada para passar para o módulo de humor
            # Incluir o título e o resumo é um bom ponto de partida.
            # O link pode ser útil para referência no output, mas não é essencial para a piada.
            texto_noticia_combinado = f"Título: {titulo}\nResumo: {resumo}"

            noticias_processadas.append(texto_noticia_combinado)
            print(f" - Notícia {i+1}: '{titulo[:50]}...' processada.") # Print para feedback

        print(f"Processamento do feed RSS concluído. {len(noticias_processadas)} notícias prontas para gerar humor.")
        return noticias_processadas

    except Exception as e:
        print(f"Ocorreu um erro ao buscar ou processar o feed RSS: {e}")
        return [] # Retorna lista vazia em caso de erro grave


# Exemplo de como você chamaria esta função na Célula 6:
# lista_noticias = buscar_noticias_economicas_rss() # Usará a URL e max_noticias padrão
# lista_noticias = buscar_noticias_economicas_rss(max_noticias=5) # Para buscar 5 notícias

In [22]:
# @title Célula 5: Módulo de Geração de Humor (Bissociação - o coração do programa!)
def gerar_piada_bisociativa(model, noticia_texto):
    """
    Gera uma piada sobre a notícia usando a teoria da bisociação de Koestler.

    Args:
        model: O objeto do modelo Gemini configurado.
        noticia_texto: O texto da notícia econômica.

    Returns:
        Uma string contendo a piada gerada e (opcionalmente) uma explicação.
    """
    if model is None:
        return "Modelo Gemini não disponível."

    # --- PROMPT ENGINEERING (CORAÇÃO DO AGENTE) ---
    prompt = f"""
    Você é um agente criativo especializado em encontrar o humor em situações sérias, usando a teoria da bisociação de Arthur Koestler.
    A bisociação é a colisão de duas ideias, contextos ou "matrizes de pensamento" habitualmente incompatíveis, que de repente se tornam consistentes de uma forma inesperada, gerando um "choque" que pode ser engraçado.

    Sua tarefa é ler a seguinte notícia econômica e encontrar duas matrizes de pensamento que possam colidir de forma humorística, seguindo a ideia de bisociação de Koestler.

    Notícia Econômica:
    "{noticia_texto}"

    Passos para gerar a piada:
    1. Identifique a "Matriz A": O conceito central, evento ou jargão principal da notícia.
    2. Pense em uma "Matriz B": Um contexto ou ideia completamente diferente e inusitado (ex: culinária, relacionamentos, contos de fadas, regras de jogos, situações do dia a dia) que tenha um ponto de contato *estrutural* ou *conceitual* abstrato com a Matriz A. Escolha um contraste forte.
    3. Gere a "Colisão/Piada": Combine as Matrizes A e B de forma inesperada e engraçada, criando uma piada curta e direta.
    4. Explique brevemente *quais* foram as Matrizes A e B que você usou e como a colisão delas cria o humor (a "faísca bisociativa") de acordo com Koestler.

    Formato da saída:
    Piada: [Sua piada aqui]
    Análise Bisociativa: [Explicação baseada em Koestler aqui]

    Gere apenas a piada e a análise opcional no formato especificado.
    """
    # --- FIM PROMPT ENGINEERING ---

    try:
        print(f"\nProcessando notícia para gerar piada bisociativa...\nNotícia: '{noticia_texto[:100]}...'") # Print para debug
        response = model.generate_content(prompt)

        # Verificar se a resposta tem texto e tratar possíveis erros de segurança
        if response.text:
             # Aqui você pode querer parsing mais robusto se seguir o formato exato
             # Por enquanto, apenas retorna o texto gerado
             return response.text
        elif response.prompt_feedback and response.prompt_feedback.block_reason:
             return f"Conteúdo bloqueado devido a configurações de segurança: {response.prompt_feedback.block_reason}"
        else:
             return "Resposta do modelo vazia ou sem texto."

    except Exception as e:
        return f"Ocorreu um erro ao gerar o conteúdo: {e}"

In [23]:
#@title Célula 6: Orquestração do Agente e Execução

if model: # Verifica se o modelo foi configurado corretamente na Célula 3
    # Definir o tópico (não é mais usado pela função RSS, mas mantemos a variável se for útil para algo no futuro)
    # Definir a quantidade de notícias a processar
    quantidade_noticias = 3 # Exemplo: processar as 3 primeiras notícias do feed

    # Passo 1: O Agente busca as notícias usando a função específica para RSS
    # Chamando a função com o NOVO nome: buscar_noticias_economicas_rss
    # Podemos usar os valores padrão definidos na Célula 4, ou especificar a quantidade
    lista_noticias = buscar_noticias_economicas_rss(max_noticias=quantidade_noticias)


    if lista_noticias:
        print("\n--- Piadas Geradas ---")
        # Passo 2: O Agente processa cada notícia para gerar humor
        for i, noticia in enumerate(lista_noticias):
            print(f"\nProcessando Notícia {i+1}/{len(lista_noticias)}:")
            # Certifique-se de que 'model' está sendo passado para a função gerar_piada_bisociativa
            piada_e_analise = gerar_piada_bisociativa(model, noticia)
            print(piada_e_analise)
            print("-" * 20) # Separador

        print("\n--- Fim da Geração ---")
    else:
        print("\nNenhuma notícia encontrada no feed RSS para processar.")

else:
    print("\nNão é possível executar a orquestração: Modelo Gemini não configurado (verifique a Célula 3 e a Célula 2 para a API Key).")

Buscando notícias do feed RSS: https://agenciabrasil.ebc.com.br/rss/economia/feed.xml
Encontradas 10 entradas no feed. Processando as primeiras 3...
 - Notícia 1: 'Redução no preço do diesel da Petrobras não chega ...' processada.
 - Notícia 2: 'Especialistas defendem diversificação nas parceria...' processada.
 - Notícia 3: 'Itaipu completa 51 anos com foco em inovação energ...' processada.
Processamento do feed RSS concluído. 3 notícias prontas para gerar humor.

--- Piadas Geradas ---

Processando Notícia 1/3:

Processando notícia para gerar piada bisociativa...
Notícia: 'Título: Redução no preço do diesel da Petrobras não chega ao IPCA
Resumo: <p><p style="text-align: c...'
Piada: A Petrobras reduz o diesel, mas o IPCA mal sente. É como se a gente estivesse numa dieta de sorvete: a sorveteria baixa o preço, mas a balança insiste em dizer que nada mudou.

Análise Bisociativa: A Matriz A é a situação econômica da redução do diesel não se refletindo no IPCA, um problema real e frustra